# Import

In [1]:
#   import packages
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.objects as so
import networkx as nx
import pickle
import functools

from ema_workbench import (
    Model,
    Policy,
    Scenario,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
    util,
    ScalarOutcome,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.em_framework.optimization import ArchiveLogger, EpsilonProgress
from ema_workbench.em_framework import parameters_from_csv
from ema_workbench.em_framework.evaluators import BaseEvaluator
from ema_workbench.analysis import parcoords
from ema_workbench.analysis import prim
from ema_workbench import Samplers

Then, the model is loaded, and logging is set.

In [2]:
dike_model, planning_steps = get_model_for_problem_formulation(5)
ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

First, the results of the policy discovery is loaded.

In [3]:
pareto_policies = pd.read_pickle(r'../generated_datasets/initial_Pareto_policies.pkl')

Then, the policies from the analysis are loaded into the model.

In [4]:
policies = []
for row_number in range(pareto_policies.shape[0]):
    policies.append(
        Policy(name=row_number, **pareto_policies.iloc[row_number, :-5].to_dict())
    )

General settings of the model are set.

In [5]:
n_scenarios = int(10_000)

Then, run the model with the robust policies as input, and generate results over a lot of scenarios

In [ ]:
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=n_scenarios,
                                            policies=policies,
                                            uncertainty_sampling=Samplers.LHS
                                            )

with open(r'../generated_datasets/policy_uncertainty_test.pkl', 'wb') as file_pi:
    pickle.dump(results, file_pi)

[MainProcess/INFO] pool started with 12 workers
[MainProcess/INFO] performing 10000 scenarios * 45 policies * 1 model(s) = 450000 experiments
  0%|                                  | 163/450000 [00:16<10:34:08, 11.82it/s]

The results are then loaded in for analysis

In [6]:
scenario_results = pd.read_pickle(r'../generated_datasets/policy_uncertainty_test.pkl')
experiments, outcomes = scenario_results
experiments

,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,scenario,policy,model
0,123,114.999161,1.5,0.097214,267.026880,10.0,0.534503,41.222939,1.5,0.074974,...,0,3,0,0,0,0,0,0,0,dikesnet
1,12,114.507209,10.0,0.539099,117.390183,1.5,0.810281,209.314732,1.0,0.011289,...,0,3,0,0,0,0,0,1,0,dikesnet
2,93,67.728956,1.0,0.518512,105.078120,10.0,0.700256,89.773865,10.0,0.407997,...,0,3,0,0,0,0,0,2,0,dikesnet
3,76,189.885408,10.0,0.381116,147.348388,1.0,0.320509,329.767171,1.0,0.035767,...,0,3,0,0,0,0,0,3,0,dikesnet
4,105,220.684581,10.0,0.822199,341.672453,10.0,0.331055,210.102760,10.0,0.801983,...,0,3,0,0,0,0,0,4,0,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224995,31,289.668313,1.0,0.052410,99.791409,1.5,0.475975,117.906512,1.5,0.997902,...,0,2,0,0,0,0,0,4995,44,dikesnet
224996,69,314.054116,1.0,0.103507,181.102471,1.5,0.875119,69.791147,1.0,0.342026,...,0,2,0,0,0,0,0,4996,44,dikesnet
224997,12,199.471980,1.0,0.020007,118.888179,1.0,0.490355,325.124367,10.0,0.750680,...,0,2,0,0,0,0,0,4997,44,dikesnet
224998,65,319.848416,1.5,0.459135,97.898038,1.5,0.945095,327.164295,1.5,0.325334,...,0,2,0,0,0,0,0,4998,44,dikesnet


Make a dataframe column for every timestep of the results.

In [7]:
policies_all_scenarios = pd.DataFrame()
for key, value in outcomes.items():
    temp_df = pd.DataFrame(value, columns=[key + ' '+ str(x) for x in range(len(value[0]))])
    policies_all_scenarios = pd.concat([policies_all_scenarios, temp_df], axis=1)

Add experiments input to the outputs for analysis

In [8]:
experiments_and_results = pd.concat([experiments,policies_all_scenarios], axis=1)

with open(r'../generated_datasets/policy_uncertainty_experiments_results.pkl', 'wb') as file_pi:
    pickle.dump(experiments_and_results, file_pi)

Combine the variables of different dikes and times

In [9]:
def combine_columns(dataframe, name):
    combined_columns = []
    for x in dataframe.columns:
        if name in x:
            combined_columns.append(x)

    dataframe['Total ' + str(name)] = dataframe[combined_columns].sum(axis=1)

In [10]:
combine_columns(policies_all_scenarios, 'Expected Annual Damage')
combine_columns(policies_all_scenarios, 'Dike Investment Costs')
combine_columns(policies_all_scenarios, 'Expected Number of Deaths')
combine_columns(policies_all_scenarios, 'RfR Total Costs')
combine_columns(policies_all_scenarios, 'Expected Evacuation Costs')

Generate statistics

In [11]:
aggregated_outcomes = pd.concat([experiments[['scenario','policy']],policies_all_scenarios.iloc[:,-5:]], axis=1)

stats_df = pd.DataFrame()
for column in aggregated_outcomes.iloc[:,2:].columns:
    stats_df[column + ' mean'] = aggregated_outcomes.groupby('policy').agg({column:['mean']})
    stats_df[column + ' std'] = aggregated_outcomes.groupby('policy').agg({column:['std']})

with open(r'../generated_datasets/policy_uncertainty_aggregated.pkl', 'wb') as file_pi:
    pickle.dump(aggregated_outcomes, file_pi)